In [28]:
import pandas as pd
import time
import websocket
import json, pprint, talib 
import csv
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

from binance.client import Client
from binance.enums import *
from coinmarketcapapi import CoinMarketCapAPI, CoinMarketCapAPIError

from pytrends.request import TrendReq as UTrendReq
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates

headers = {
    'authority': 'trends.google.com',
    'sec-ch-ua': '"Chromium";v="94", " Not A;Brand";v="99", "Opera";v="80"',
    'accept': 'application/json, text/plain, */*',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36 OPR/80.0.4170.63',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://trends.google.com/trends/explore?q=mirror%20protocol',
    'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    'cookie': '__utmc=10102256; __utma=10102256.830189433.1636580671.1637612468.1637615437.6; __utmz=10102256.1637615437.6.4.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); __utmt=1; __utmb=10102256.9.9.1637615792660; CONSENT=YES+GB.en-GB+V9+BX; SEARCH_SAMESITE=CgQImZMB; HSID=A0PbYKwoJWHkP2cLJ; SSID=ATw-D_CgCgaCnOMxq; APISID=hng8MXyBWur_w5fC/AurWCgPbsYDEtwkWm; SAPISID=vSGlu5opiC_3-Dav/Ak0loYtHrjyKusKXH; __Secure-1PAPISID=vSGlu5opiC_3-Dav/Ak0loYtHrjyKusKXH; __Secure-3PAPISID=vSGlu5opiC_3-Dav/Ak0loYtHrjyKusKXH; OGPC=19025836-2:; __Secure-1PSIDCC=AJi4QfHJF0V84yFFydIeqwcTavYgTxPKZTm4r-j92P2vuN691-chSFUrnLeYq4ccPLlok1ebJg; SID=EAgztJro1wlVbGdfzKYVqqxGYz9eHXx7y-0pWf9ljDCqShG7I4iq_DUrjYYzifOFgAkPdg.; __Secure-1PSID=EAgztJro1wlVbGdfzKYVqqxGYz9eHXx7y-0pWf9ljDCqShG7Arm0dTwN1pVkMU_W42hVkA.; __Secure-3PSID=EAgztJro1wlVbGdfzKYVqqxGYz9eHXx7y-0pWf9ljDCqShG7J8Mh2faUlqm65EfZ7UsPCQ.; NID=511=esdr8y2puX552h2JBc-TknP1obzsr3qLgSA0yeAV7DnfwAVYC251PzaCr4yYLsm0EyMjI7L7neOP6JTtPSHjnHgitlzfoWrzX7xcm8n8hB6wIeLNCMeECr-IhEQs3-ocHCofs6pbgR6LwfrvhIvF9WCClueXUX4FKGD3CQ14xkM9ikLu44rqvJjgYtyXfcwcFl3qpWZOKGfuY4Jz_Molfk7bNBlNNaNUe0CeMlei3I4dIkHbEBlAA6LW8vFhnIZyMWhCd2lVkAiy0c_FOaUUp7iCKPUj6RD-ddXs7X_j7X5-2DRkXkUZI5Cl8d-ZyyPWDGcnOtvPGwBiTGQ8o7tRxRMRMjjR_cnrR9TyMUIdYJ7F7cJmTl9GUqgZ-spxHWzUU0hlgmrdLErIBsTdXA; 1P_JAR=2021-11-22-21; SIDCC=AJi4QfHp9yLbArD3EA6819JHREwwDTB9vLz-bkPMpDjk7VY2Zh26Zb4JTNu7jl3FkmbY6CEZeL18; __Secure-3PSIDCC=AJi4QfEct8ZkJckjPrd7cpR-xm748VcgUsitncJ7DEkOKMdTbMOZSwaZbCxEyvinm2nCvqQ-3rQ3',
}

GET_METHOD='get'
class TrendReq(UTrendReq):
    def _get_data(self, url, method=GET_METHOD, trim_chars=0, **kwargs):
        return super()._get_data(url, method=GET_METHOD, trim_chars=trim_chars, headers=headers, **kwargs)


cmc = CoinMarketCapAPI('a01ace6b-f43c-439e-9fca-dc7f73e2a648')
api_key = 'ihvXWbHvvDrYuroIJOS9ANW6X7tv5OQSYibLLloKyIGui0w93Q5yUJOBZclnNL5D'
api_secret = 'uzmKBJ3lmVS4TsTl2AOHVJOYTFfLM5qrg00vAqUyTYbSjiy7BDPlTpaq2Oy7qdo2'
client = Client(api_key, api_secret)

def get_all_tokens():
    #filter out leverage tokens, non usd tokens, 
    all_pairs = pd.DataFrame(client.get_ticker())
    relev = all_pairs[all_pairs.symbol.str.endswith('USDT')]
    non_lev_list = relev[ ~( (relev.symbol.str.contains('UP')) | (relev.symbol.str.contains('DOWN')) | (relev.symbol.str.contains('BEAR')) | (relev.symbol.str.contains('BULL')) ) ]
    #filter out stable coins
    non_stable_rm = non_lev_list[~all_pairs.symbol.str.startswith('USD')]
    #remove BCHSV, PAX, STRAT, XZC
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'BCHSVUSDT'].index )
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'PAXUSDT'].index ) # dollar value
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'STRATUSDT'].index )
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'XZCUSDT'].index )
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'AUDUSDT'].index ) # non crypto
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'EURUSDT'].index ) # non crypto
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'GBPUSDT'].index ) # non crypto
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'BCCUSDT'].index ) # BitConnect scam
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'BCHABCUSDT'].index ) # Bitcoin Cash ABC (BCHA) Rebrand to eCash (XEC)
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'BUSDUSDT'].index ) # Binance USD
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'BTGUSDT'].index ) # Bitcoin Gold
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'ERDUSDT'].index ) # Elrond (ERD) Mainnet & Token Swap to Elrond Gold (EGLD) 
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'LENDUSDT'].index ) # LEND Token Swap to AAVE
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'MCOUSDT'].index ) # MCO Token unsupported

    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'GXSUSDT'].index ) # Find a way to replace lookup
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'IOTAUSDT'].index ) # Find a way to replace lookup

    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'NPXSUSDT'].index ) # Pundi X old -> PUNDIX
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'PAXGUSDT'].index ) # Pax Gold Binance Completes VEN Mainnet Swap to VET
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'VENUSDT'].index ) # Binance Completes VEN Mainnet Swap to VET 
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'TUSDUSDT'].index ) # stable coin TrueUSD 
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'STORMUSDT'].index ) #Storm (STORM) Token Swap to STMX  NPXS	 
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'SUSDUSDT'].index ) # usd coin sUSD
    # symbols = symbols.replace('GXS', 'GXC')
    # symbols = symbols.replace('IOTA', 'MIOTA')


    #sorted = non_stable_rm.sort_values(by='priceChangePercent', ascending=False)
    sorted = non_stable_rm.sort_values(by='symbol', key=lambda x: x.str.split('USDT').str[0])

    token_tickers = sorted
    symbols=token_tickers['symbol'].str.slice(0,-4)
    # symbols = symbols.replace('GXS', 'GXC')
    # symbols = symbols.replace('IOTA', 'MIOTA')
    symbolslist = ",".join(symbols)

    cmcinfo = cmc.cryptocurrency_info(symbol=symbolslist)
    names_info = cmcinfo.data
    names = []
    for ind in names_info:
        names.append(names_info[ind]['name'].replace('[', '').replace(']', '').replace('(', '') .replace(')', ''))
        
    sorted.insert(1, "name", names)

    alltokens = sorted[['symbol', 'name']]
    
    #sorted['names'] = names

    return alltokens.reset_index().rename({'index': 'Binance Index'}, axis=1)

#format (symbol, interval, 120 mins ago, now) or (symbol, interval, start date, end date) "1 Dec, 2017"
def getminutedata(symbol, interval, start, end):
    if(end == 'now'):
        frame = pd.DataFrame(client.get_historical_klines(symbol,
                                                        interval,
                                                        start + ' min ago UTC'))
    else:
        frame = pd.DataFrame(client.get_historical_klines(symbol, 
                                                        interval, 
                                                        start, 
                                                        end))
    frame = frame.iloc[:,:6]                                                    
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index, unit= 'ms')
    frame = frame.astype(float)
    return frame.reset_index()

def get_trends_3(coin, start, end):
    pytrends = TrendReq(hl='en-GB')
    data = pytrends.get_historical_interest(coin, 
                                    year_start=start[3], 
                                    month_start=start[2], 
                                    day_start=start[1], 
                                    hour_start=start[0], 
                                    year_end=end[3], 
                                    month_end=end[2], 
                                    day_end=end[1], 
                                    hour_end=start[0], 
                                    cat=0, 
                                    geo='', 
                                    gprop='', 
                                    sleep=0)
    data = data.drop('isPartial', axis=1)
    return data


In [8]:
allCoin_df = get_all_tokens()

C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_18088/1011553277.py:50: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  non_stable_rm = non_lev_list[~all_pairs.symbol.str.startswith('USD')]


In [9]:
allCoin_array = allCoin_df.to_numpy()
allCoin_list = []
for x in range(len(allCoin_array)-1):
    allCoin_list.append([allCoin_array[x][1], [allCoin_array[x][2]]])

coin = allCoin_list[1]



dayDelta = 0
monthDelta = -1
dt_end      = datetime.now()
dt_start    = (datetime.now() + relativedelta(months=monthDelta) + timedelta(days=dayDelta))
dt_ar_end   = list(map(int,   dt_end.strftime("%d-%m-%Y-%H-%M-%S").split('-') ))
dt_ar_start = list(map(int, dt_start.strftime("%d-%m-%Y-%H-%M-%S").split('-') ))
end         = [dt_ar_end[3],  dt_ar_end[0],  dt_ar_end[1],  dt_ar_end[2]]
start       = [dt_ar_end[3],dt_ar_start[0],dt_ar_start[1],dt_ar_start[2]]

# start = [0,21,11,2021]
# end = [0,21,11,2021]
#dt_start = datetime.datetime(start[3], start[2], start[1])
#dt_end   = datetime.datetime(  end[3],   end[2],   end[1])

time_delta = (dt_end - dt_start)
total_seconds = time_delta.total_seconds()
minutes = total_seconds/60
time_var = str(int(minutes))

COIN = getminutedata(coin[0], '1m', time_var, 'now')  
HYPE = get_trends_3(coin[1], start, end)

KeyboardInterrupt: 

In [ ]:
HYPE.index.names = ['Date']
COIN.index.names = ['Date']
name = coin[0][:4]
COIN =  COIN.rename(columns={"AAVEUSDT": coin[0][:4]+' ($)'})

print(COIN)
print(name)
#COIN = COIN.merge(HYPE, how='outer', on='Date').ffill()

#COIN.to_pickle("./COIN.pkl")
#COIN.to_csv("./COIN.csv")

                     Close
Date                      
2021-10-27 15:31:00  350.9
2021-10-27 15:32:00  349.7
2021-10-27 15:33:00  351.2
2021-10-27 15:34:00  350.6
2021-10-27 15:35:00  350.8
...                    ...
2021-11-27 15:27:00  238.5
2021-11-27 15:28:00  238.7
2021-11-27 15:29:00  238.3
2021-11-27 15:30:00  238.5
2021-11-27 15:31:00  238.6

[44641 rows x 1 columns]
AAVE


In [8]:
# from IPython.display import display
# pd.set_option('display.max_rows', 400)

# COIN1 = pd.read_pickle("./COIN.pkl")
# COIN2 = pd.read_csv('COIN.csv', index_col=0)
# #COIN
# #HYPE.index = pd.to_datetime(HYPE.index, unit= 'ms')

# #display(COIN1)

# #print(COIN1['Aave'])

# COIN2.ffill().plot(subplots=True)
# plt.show()

# COIN1

# COIN2

# import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize'] = [10, 5]
# def plot_coins(coin, HYPE, CLOSE):
#     plt.figure(coin[1][0])
#     plt.plot(CLOSE, color="green")
#     plt.xlabel("Date & Time")
#     plt.ylabel("Value $")
#     plt2=plt.twinx()
#     plt2.plot(HYPE)
#     #plt2.plot(HYPE.diff(periods=1))
#     #plt2.plot(HYPE.diff(periods=2))
#     plt2.set_ylabel("Hype %")
#     plt2.legend(['Close price'])
#     plt.legend(['Buy Indicator', 'Rolling indicator 5 + 20%', 'Rolling indicator 5', 'differential 1', 'differential 2'])
#     plt.title(coin[1])
#     plt.show(block=False)

# plot_coins(coin, HYPE, VALUE['Close'])

In [10]:
all_pairs = pd.DataFrame(client.get_ticker())
relev = all_pairs[all_pairs.symbol.str.endswith('USDT')]
non_lev_list = relev[ ~( (relev.symbol.str.contains('UP')) | (relev.symbol.str.contains('DOWN')) | (relev.symbol.str.contains('BEAR')) | (relev.symbol.str.contains('BULL')) ) ]
#filter out stable coins
non_stable_rm = non_lev_list[~all_pairs.symbol.str.startswith('USD')]
#remove BCHSV, PAX, STRAT, XZC
non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'BCHSVUSDT'].index )
non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'PAXUSDT'].index ) # dollar value
non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'STRATUSDT'].index )
non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'XZCUSDT'].index )
non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'AUDUSDT'].index ) # non crypto
non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'EURUSDT'].index ) # non crypto
non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'GBPUSDT'].index ) # non crypto
non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'BCCUSDT'].index ) # BitConnect scam
non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'BCHABCUSDT'].index ) # Bitcoin Cash ABC (BCHA) Rebrand to eCash (XEC)
non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'BUSDUSDT'].index ) # Binance USD
non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'BTGUSDT'].index ) # Bitcoin Gold
non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'ERDUSDT'].index ) # Elrond (ERD) Mainnet & Token Swap to Elrond Gold (EGLD) 
non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'LENDUSDT'].index ) # LEND Token Swap to AAVE
non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'MCOUSDT'].index ) # MCO Token unsupported

non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'GXSUSDT'].index ) # Find a way to replace lookup
non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'IOTAUSDT'].index ) # Find a way to replace lookup

non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'NPXSUSDT'].index ) # Pundi X old -> PUNDIX
non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'PAXGUSDT'].index ) # Pax Gold Binance Completes VEN Mainnet Swap to VET
non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'VENUSDT'].index ) # Binance Completes VEN Mainnet Swap to VET 
non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'TUSDUSDT'].index ) # stable coin TrueUSD 
non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'STORMUSDT'].index ) #Storm (STORM) Token Swap to STMX  NPXS	 
non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'SUSDUSDT'].index ) # usd coin sUSD
# symbols = symbols.replace('GXS', 'GXC')
# symbols = symbols.replace('IOTA', 'MIOTA')


#sorted = non_stable_rm.sort_values(by='priceChangePercent', ascending=False)
sorted = non_stable_rm.sort_values(by='symbol', key=lambda x: x.str.split('USDT').str[0])

token_tickers = sorted
symbols=token_tickers['symbol'].str.slice(0,-4)
# symbols = symbols.replace('GXS', 'GXC')
# symbols = symbols.replace('IOTA', 'MIOTA')
symbolslist = ",".join(symbols)
cmcinfo = cmc.cryptocurrency_info(symbol=symbolslist)
names_info = cmcinfo.data
names = []
for ind in names_info:
    names.append(names_info[ind]['name'].replace('[', '').replace(']', '').replace('(', '') .replace(')', ''))

names_info = pd.DataFrame(names_info).T
names_info.insert(1, "ticker", list(sorted['symbol']))
names_info = names_info.rename({'id': 'cmc_id'}, axis=1)
alltokens = names_info[['cmc_id', 'symbol','ticker','name','category','date_added','tags', 'twitter_username', 'self_reported_tags', 'subreddit']]
alltokens = alltokens.reset_index()
alltokens = alltokens.drop(columns=['index'])
alltokens['id'] = alltokens.index
#sorted['names'] = names
#alltokens.reset_index().rename({'index': 'Binance Index'}, axis=1)

C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_18088/4285169069.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  non_stable_rm = non_lev_list[~all_pairs.symbol.str.startswith('USD')]


In [11]:
import os
import sqlite3 as sl
from sqlite3 import Error

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sl.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
        return conn
    except Error as e:
        print(e)
    return conn

def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

db_file = 'crypto_database.db'

In [12]:
#['symbol','ticker','name','category','date_added','tags', 'twitter_username', 'self_reported_tags', 'subreddit' ]
sql_create_crypto_table = """CREATE TABLE IF NOT EXISTS crypto (
                                id integer ,
                                name text NOT NULL,
                                symbol text,
                                category text,
                                date_added text,
                                name_searchable text,
                                tags text,
                                twitter_username text,
                                self_reported_tags text,
                                subreddit text,
                                cmc_id text,
                                artifact_id integer,
                                PRIMARY KEY(id),
                                FOREIGN KEY (artifact_id) REFERENCES artifact (id)
                            );"""

sql_create_artifact_table = """CREATE TABLE IF NOT EXISTS artifact (
                            id integer,
                            ticker text NOT NULL UNIQUE,
                            symbol text,
                            name text NOT NULL,
                            crypto_id integer,
                            PRIMARY KEY(id),
                            FOREIGN KEY (crypto_id) REFERENCES crypto (id)
                        );"""

sql_create_artifact_price_table = """CREATE TABLE IF NOT EXISTS artifact_price (
                    id integer,
                    date datetime NOT NULL,
                    open decimal NOT NULL,
                    high decimal NOT NULL,
                    low decimal NOT NULL,
                    close decimal NOT NULL,
                    volume integer,
                    artifact_id integer,
                    PRIMARY KEY(id),
                    FOREIGN KEY (artifact_id) REFERENCES artifact (id)
                );"""

sql_create_artifact_trend_table = """CREATE TABLE IF NOT EXISTS artifact_price (
                    id integer,
                    date datetime NOT NULL,
                    hype decimal NOT NULL,
                    artifact_id integer,
                    PRIMARY KEY(id),
                    FOREIGN KEY (artifact_id) REFERENCES artifact (id)
                );"""

conn = create_connection(db_file)

# create tables
if conn is not None:
    # create crypto table
    create_table(conn, sql_create_crypto_table)
    # create artifact table
    create_table(conn, sql_create_artifact_table)
    # create artifact_price table
    create_table(conn, sql_create_artifact_price_table)
else:
    print("Error! cannot create the database connection.")

In [14]:
conn = sl.connect(db_file)

crypto_table = alltokens[['id', 'name', 'symbol', 'category', 'date_added', 'name'           , 'tags', 'twitter_username', 'self_reported_tags', 'subreddit', 'cmc_id']]
crypto_table.columns  =  ['id', 'name', 'symbol', 'category', 'date_added', 'name_searchable', 'tags', 'twitter_username', 'self_reported_tags', 'subreddit', 'cmc_id']



#crypto_table['type'] = crypto_table['type'].map(lambda type: type.str.join(','))
#crypto_table['type'] = crypto_table['type'].map(lambda type: (",".join(type)))

crypto_table['tags'] = crypto_table['tags'].fillna('0').map(lambda type: (','.join(map(str, type))))
crypto_table['self_reported_tags'] = crypto_table['self_reported_tags'].fillna('0').map(lambda type: (','.join(map(str, type))))


artifact_table = alltokens[['id', 'ticker', 'symbol', 'name']]

crypto_table

artifact_id_mapper = pd.Series(artifact_table.id.values, index=crypto_table.name).to_dict()

crypto_table['artifact_id'] = pd.Series(crypto_table['name']).map(artifact_id_mapper)

crypto_id_mapper = pd.Series(crypto_table.id.values, index=crypto_table.name).to_dict()

artifact_table['crypto_id'] = pd.Series(artifact_table['name']).map(crypto_id_mapper)


# crypto_table.to_sql("crypto", conn, if_exists="append", index=False)
# artifact_table.to_sql("artifact", conn, if_exists="append", index=False)

C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_18088/3843962014.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  artifact_table['crypto_id'] = pd.Series(artifact_table['name']).map(crypto_id_mapper)


In [38]:
dayDelta = 0
monthDelta =-3
dt_end      = datetime.now()
dt_start    = (datetime.now() + relativedelta(months=monthDelta) + timedelta(days=dayDelta))
dt_ar_end   = list(map(int,   dt_end.strftime("%d-%m-%Y-%H-%M-%S").split('-') ))
dt_ar_start = list(map(int, dt_start.strftime("%d-%m-%Y-%H-%M-%S").split('-') ))
end         = [dt_ar_end[3],  dt_ar_end[0],  dt_ar_end[1],  dt_ar_end[2]]
start       = [dt_ar_end[3],dt_ar_start[0],dt_ar_start[1],dt_ar_start[2]]

# start = [0,21,11,2021]
# end = [0,21,11,2021]
#dt_start = datetime.datetime(start[3], start[2], start[1])
#dt_end   = datetime.datetime(  end[3],   end[2],   end[1])

time_delta = (dt_end - dt_start)
total_seconds = time_delta.total_seconds()
minutes = total_seconds/60
time_var = str(int(minutes))

In [1]:
from tqdm.notebook import tqdm
artifact_pricing_dfs = []

for artifact_id in tqdm(artifact_table['id']):
    try:
        artifact_pricing_df = getminutedata(artifact_table.iloc[artifact_id]['ticker'], 
                                        '15m', 
                                        time_var, 
                                        'now') 
        artifact_pricing_df['artifact_id'] = artifact_id
        artifact_pricing_dfs.append(artifact_pricing_df)
    except:
        pass
artifact_price_table = pd.concat(artifact_pricing_dfs)
artifact_price_table.columns = ['time', 'hype', 'high', 'low', 'close', 'volume', 'artifact_id']

artifact_price_table.reset_index(inplace=True)
artifact_price_table['id'] = artifact_price_table.index
artifact_price_table.to_sql("artifact_price", conn, if_exists="append", index=False)


NameError: name 'artifact_table' is not defined

In [36]:
artifact_trend_dfs = []

for artifact_id in tqdm(artifact_table['id']):
    try:
        artifact_trend_df = get_trends_3(artifact_table.iloc[artifact_id]['name_searchable'], start, end)
        artifact_trend_df['artifact_id'] = artifact_id
        artifact_trend_dfs.append(artifact_trend_df)
    except:
        pass
artifact_trend_table = pd.concat(artifact_trend_dfs)
artifact_trend_table.columns = ['time', 'hype', 'artifact_id']

artifact_trend_table.reset_index(inplace=True)
artifact_trend_table['id'] = artifact_price_table.index
artifact_trend_table.to_sql("artifact_trend", conn, if_exists="append", index=False)

  0%|          | 0/285 [00:00<?, ?it/s]

KeyError: 'ticker'

In [37]:
crypto_table

,id,name,symbol,category,date_added,name_searchable,tags,twitter_username,self_reported_tags,subreddit,cmc_id,artifact_id
0,0,1inch Network,1INCH,token,2020-12-25T00:00:00.000Z,1inch Network,"decentralized-exchange,defi,amm,binance-smart-...",1inch,0,1inch,8104,0
1,1,Aave,AAVE,token,2020-10-02T00:00:00.000Z,Aave,"defi,dao,yield-farming,binance-smart-chain,thr...",AaveAave,0,Aave_Official,7278,1
2,2,AC Milan Fan Token,ACM,token,2021-02-22T00:00:00.000Z,AC Milan Fan Token,"sports,fan-token,binance-launchpad",Chiliz,0,,8538,2
3,3,Cardano,ADA,coin,2017-10-01T00:00:00.000Z,Cardano,"mineable,dpos,pos,platform,research,smart-cont...",cardano,Communications & Social Media,cardano,2010,3
4,4,Ambire AdEx,ADX,token,2017-07-01T00:00:00.000Z,Ambire AdEx,"marketing,smart-contracts,substrate,polkadot,b...",AmbireAdEx,"Marketing,Services,DeFi,Staking,Ethereum Ecosy...",AdEx,1768,4
...,...,...,...,...,...,...,...,...,...,...,...,...
280,280,Yield Guild Games,YGG,token,2021-06-28T00:00:00.000Z,Yield Guild Games,"collectibles-nfts,gaming,entertainment,metaver...",YieldGuild,0,,10688,280
281,281,Zcash,ZEC,coin,2016-10-29T00:00:00.000Z,Zcash,"mineable,pow,equihash,medium-of-exchange,priva...",electriccoinco,0,zec,1437,281
282,282,Horizen,ZEN,coin,2017-06-01T00:00:00.000Z,Horizen,"mineable,pow,medium-of-exchange,enterprise-sol...",horizenglobal,"DPoS,PoS,PoW,Equihash,Education,Medium of Exch...",Horizen,1698,282
283,283,Zilliqa,ZIL,coin,2018-01-25T00:00:00.000Z,Zilliqa,"mineable,platform,payments,smart-contracts,bin...",zilliqa,"Collectibles & NFTs,Sharding,Smart Contracts",zilliqa,2469,283
